In [1]:
import os
import time
import re
import glob
import pandas as pd
import requests
        
def get_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    names = []
    for b in blobs: names.append(b.name)
    return names

def upload_to_output(path, bucket_name, folder_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(folder_name + '/' + path.split('/')[-1])
    blob.upload_from_filename(path)
    
def stack_tweets(new_records, existing_records):
    captured_tokens = list(existing_records.id.unique())
    response_tokens = list(new_records.id.unique())
    already_captured = [x for x in response_tokens if x in captured_tokens]
    
    x = len(captured_tokens)
    y = len(response_tokens)
    z = len(already_captured)
    
    existing_records = existing_records.append(new_records, ignore_index=False)
    
    print('Existing tokens: ', x, 'Response tokens: ', y,
          'Already_captured: ', z, 'Efficiency Rate: ', str(round(z/y,2)*100)+'%')
    return existing_records

def searchRTOfTweetIDQuery(tweet_id):
    return 'retweets_of_tweet_id:'+str(tweet_id)

    
class TwitterAPI():
    def __init__(self, bearer_token, consumer_key, consumer_secret, access_token, access_token_secret):
        import requests
        self.client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)
        
        
        self.results = {}
        
    def get_tweets(self, keyword, max_results):
        query = keyword
        from datetime import datetime
        ts = str(datetime.now())
        reference_id = ts+'__'+keyword
        if max_results <= 100:
            tweets = self.client.search_recent_tweets(query=query, 
                                                tweet_fields=['id', 'author_id', 'conversation_id',
                                                              'in_reply_to_user_id', 'created_at',
                                                              'text', 'public_metrics'],
                                                 max_results=max_results)
            tweets_dict = tweets.json() 
            tweets_data = tweets_dict['data'] 
            df = pd.json_normalize(tweets_data)
            df = df.set_index('id')

            self.results[reference_id] = df

            return df
        
        else:
            
            count = 0
            done = False
            
            tweets = self.client.search_recent_tweets(query=query, 
                                                    tweet_fields=['id', 'author_id', 'conversation_id',
                                                                  'in_reply_to_user_id', 'created_at',
                                                                  'text', 'public_metrics'],
                                                     max_results=100)
            tweets_dict = tweets.json() 
            tweets_data = tweets_dict['data'] 
            df = pd.json_normalize(tweets_data)
            existing_records = df.set_index('id')
            
            count = len(df)
            
            while not done:
                
                tweets = self.client.search_recent_tweets(query=query, 
                                                    tweet_fields=['id', 'author_id', 'conversation_id',
                                                                  'in_reply_to_user_id', 'created_at',
                                                                  'text', 'public_metrics'],
                                                     max_results=100)
                tweets_dict = tweets.json() 
                tweets_data = tweets_dict['data'] 
                new_records = pd.json_normalize(tweets_data)
                new_records = df.set_index('id')

                x = len(existing_records)
                y = len(new_records)

                existing_records = existing_records.append(new_records, ignore_index=False)
                
                added = len(existing_records) - x
                
                eff = round(added/y,2)
                
                print('Existing tokens: ', x, 'Response tokens: ', y, 'Already_captured: ',
                      y-added, 'Efficiency Rate: ', str(eff*100)+'%')
                
                count+=added
                
                done = count >= max_results
                
            df = existing_records
            self.results[reference_id] = df
        
        return df
    
    def aggregate_results(self):
        searches = list(self.results.keys())

        src = []
        for x in searches:
            src.append(api.results[x])

        results = pd.concat(src, ignore_index=False)
        
        return results.copy()

In [2]:
# Create Twitter API Client
import os

try: import tweepy
except ModuleNotFoundError: 
    !pip install tweepy
    import tweepy
    
import pandas as pd
pd.options.display.max_colwidth = 280


# consumer_key = '1r00ftJDzKTcaLCGgb0WWlCak'
# consumer_secret = 'ligckZVYi8ygX0Rs4HhcVMjnQ0NB3vFG8cG9QMQXyFWei1sO2X'
# access_token = '1547339723798093829-2w6gIpXLZf2953rkyZbbN95dkQcs55'
# access_token_secret = 'RUSGsMrqpJIeKbQhCGn5DwAZDpV4EAgxZvudKrQRhXkq6'
# bearer_token = 'AAAAAAAAAAAAAAAAAAAAAG7lfwEAAAAAn3AnJpgxBjK2FmzPlulBSH8440U%3DEiB4dx0ggR5GnRZqmgDS6oHd8iqD58dDAqfLzZAV7tZWNBfRHX'

consumer_key = "afxygKWvjyHeYqK2VOudagVF5"
consumer_secret = "s3aiZTpaWJXsqkqzxdhetnGaXSoXyJo5fay0PvaRbe7azGUTdB"
access_token = "133488849-J5PyZGIBiWaZ91gd4EaL9Ni0yaCeUUeevA3pUk3j"
access_token_secret = "39wEmtno1NucvjkT5GrBupwQstmHaPukj3KRFZj48EW31"
bearer_token = "AAAAAAAAAAAAAAAAAAAAANgJegEAAAAA7apqttgkq4Do4dGRoxfB0MD6D6w%3DGTFCK7VJJjA4fR0d0k9XCsrV6h7D36pmLyq6g5Vmnw9ZtZxT7k"

creds = {'consumer_key':consumer_key,
'consumer_secret':consumer_secret,
'access_token' : access_token,
'access_token_secret' : access_token_secret,
'bearer_token' : bearer_token} 

api = TwitterAPI(creds['bearer_token'], 
                       creds['consumer_key'], 
                       creds['consumer_secret'], 
                       creds['access_token'], 
                       creds['access_token_secret'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 4.5 MB/s eta 0:00:00


In [3]:
# FIRST RUN - CREATE USER TABLE
import pandas as pd
df = pd.read_csv('gs://sw-airlines-data-hub/data/processed/sw-airlines-tweets-base.csv')
user_df = pd.read_csv('gs://sw-airlines-data-hub/data/users/user_id.csv')
user_df = user_df.set_index('id')
recorded_users = list(user_df.index.unique())

user_id = df.drop(columns=['id', 'text', 'conversation_id','created_at',
                           'in_reply_to_user_id','public_metrics.retweet_count',
                           'public_metrics.reply_count','public_metrics.like_count','public_metrics.quote_count'])
user_id = user_id.set_index('author_id')
user_id.to_csv('gs://sw-airlines-data-hub/data/src/author_id.csv')
user_list = list(df.author_id.unique())
new_users = [x for x in user_list if x not in recorded_users]
len(new_users)


5693

In [4]:
import time
if len(new_users) > 0:
    stack = []
    dfs = []

    while user_list and len(new_users) > 0:
        time.sleep(3)
        print(len(new_users))
        while len(stack) < 99 and new_users:
            stack.append(new_users.pop())
        users = api.client.get_users(ids=list(stack), user_fields = ['id',
                                                                     'username', 
                                                                     'profile_image_url', 
                                                                     'verified', 
                                                                     'description', 
                                                                     'created_at', 
                                                                     'public_metrics'])
        try:
            user_dict = users.json() 
            user_data = user_dict['data'] 
            df = pd.json_normalize(user_data)
            dfs.append(df)
        except KeyError:
            pass
        stack = []

    result = dfs.pop()
    result = result.set_index('id')

    for x in dfs: 
        x = x.set_index('id')
        result = result.append(x, ignore_index=False)
    user_df = user_df.append(result, ignore_index=False)
    os.system('mkdir data')
    user_df.to_csv('./data/user_id.csv')
    from google.cloud import storage
    storage_client = storage.Client()
    bucket_name='sw-airlines-data-hub'
    bucket = storage_client.bucket(bucket_name)
    upload_to_output(path='data/user_id.csv', bucket_name=bucket_name, folder_name='data/users')

5693
5594
5495
5396
5297
5198
5099
5000
4901
4802
4703
4604
4505
4406
4307
4208
4109
4010
3911
3812
3713
3614
3515
3416
3317
3218
3119
3020
2921
2822
2723
2624
2525
2426
2327
2228
2129
2030
1931
1832
1733
1634
1535
1436
1337
1238
1139
1040
941
842
743
644
545
446
347
248
149
50


In [ ]:
result = dfs.pop()
result = result.set_index('id')
for x in dfs: 
    x = x.set_index('id')
    result = result.append(x, ignore_index=False)
user_df = user_df.append(result, ignore_index=False)
user_df.to_csv('data/user_id.csv')
from google.cloud import storage
storage_client = storage.Client()
bucket_name='sw-airlines-data-hub'
bucket = storage_client.bucket(bucket_name)
upload_to_output(path='data/user_id.csv', bucket_name=bucket_name, folder_name='data/users')